In [1]:
import tabulate
from pathlib import Path
import numpy as np

from typing import List, Dict
from collections import defaultdict
import warnings
import copy

warnings.filterwarnings("error")


import tabulate

log_dir = '../log'

def prettify(exps_mean, exps_cp):
    for key in metrics:
        exps_cp[key][''] = []
        for k in exps_mean[key]['']:
            k = k.replace('discriminator', 'dis')
            k = k.replace('mmd', 'MMD')
            k = k.replace('cmd', 'CMD')
            k = k.replace('_dis', ' (d)')
            k = k.replace('_', ' ')
            if k == 'cluster':
                k = 'cluster v0'
            k = k.replace('dumb', 'w/o constr.')
            exps_cp[key][''].append(k)

        for model in models:
            exps_cp[key].pop(model)
            for i, mean in enumerate(exps_mean[key][model]):#, exps_std[model])):
                if model == 'mlp_ring':
                    model_ = 'JS-GAN, 2d ring'
                elif model == 'mlp_js':
                    model_ = 'JS-GAN, 2d grid'
                elif model == 'mlp_wass':
                    model_ = 'WGAN, grid'
                elif model == 'mlp_js3':
                    model_ = 'JS-GAN, 3d grid'
                exps_cp[key][model_].append(mean)

In [2]:
models = ['mlp_wass', 'mlp_js']#, 'mlp_ring']
targets = ["Discriminator", "Prior"]

step_sizes = [0.0001, 0.0001]#, 0.001]

features = {
    "dumb": [""],
    "discriminator": [""],
    "cluster": {
         "": ["", "_dis"],
        "_v1": ["", "_dis"],
        "_v2": {
            "": ["", "_dis"],
            # "_lin": ["", "_dis"],
            # "_quad": ["", "_dis"],
        },
        # "_v3": ["", "_dis"]
    },
    "mmd": {
            "": ["", "_dis"],
            # "_lin": [""], #"_dis"],
            # "_quad": [""],#, "_dis"],
        },
    "cmd": {
            "": ["", "_dis"],
        }, 
}
metrics = ['EMD', 'HQR', 'JSD']

In [7]:
exps_mean = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
exps_std = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

# for target in targets:
#     for metric in metrics:
#         exps_mean[target][metric][''].append('GAN')

for target in targets:
    for key in metrics:
        exps_mean[target][key][''].append('GAN')
        exps_std[target][key][''].append('GAN')

for target in targets:
    for model, step_size in zip(models, step_sizes):

        path = Path(log_dir, f'dumb_feature_DiscriminatorTarget', f'{model}_{step_size}')
        for key in metrics:
            m = np.loadtxt(Path(path, f'{key}.txt'))[:, 0]
            m_std = m.std(ddof=1)
            m_mean = m.mean()

            exps_mean[target][key][model].append(m_mean)
            exps_std[target][key][model].append(m_std)

def move(subs, suff, feature):
    if isinstance(subs, List):
        for target in targets:
            for model, step_size in zip(models, step_sizes):
                for s in subs:
                    path = Path(log_dir, f'{feature}_feature_{target}Target', f'{model}{suff}{s}_{step_size}')

                    for key in metrics:
                        try:        
                            m = np.loadtxt(Path(path, f'{key}.txt'))[:, 1]
                        except (IndexError, UserWarning, OSError):
                            m = np.zeros(3)
                        m_std = m.std(ddof=1)
                        m_mean = m.mean()

                        exps_mean[target][key][model].append(m_mean)
                        exps_std[target][key][model].append(m_std)
                        if model == models[0]:
                            exps_mean[target][key][''].append(feature+path.name[len(model):-len(f'_{step_size}')])
                            exps_std[target][key][''].append(feature+path.name[len(model):-len(f'_{step_size}')])
        
        return
    elif isinstance(subs, Dict):
        for sub_key, subs_ in subs.items():
            move(subs_, suff+sub_key, feature)

for feature, subs in features.items():
    move(subs, "", feature)


5
5
5
5
5
5
3
3
3
3
3
3
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5


In [8]:
import copy

# exps_mean_cp =  copy.deepcopy(exps_mean)
# exps_std_cp =  copy.deepcopy(exps_std)

# prettify(exps_mean, exps_mean_cp)
# prettify(exps_std, exps_std_cp)

# exps_cp = copy.deepcopy(exps_mean_cp)

# for key in metrics:
#     for model in list(exps_mean_cp[key].keys())[1:]:
#         for i, (mean, std) in enumerate(zip(exps_mean_cp[key][model], exps_std_cp[key][model])):
#             exps_cp[key][model][i] = fr'${mean:.3f}({std:.3f})$'

exps_mean_cp =  copy.deepcopy(exps_mean)
exps_std_cp =  copy.deepcopy(exps_std)

for target in targets:
    prettify(exps_mean[target], exps_mean_cp[target])
    prettify(exps_std[target], exps_std_cp[target])

exps_cp = copy.deepcopy(exps_mean_cp)

for target in targets:
    for key in metrics:
        for model in list(exps_mean_cp[target][key].keys())[1:]:
            for i, (mean, std) in enumerate(zip(exps_mean_cp[target][key][model], exps_std_cp[target][key][model])):
                exps_cp[target][key][model][i] = fr'${mean:.3f}({std:.3f})$'

In [9]:
print(tabulate.tabulate(exps_cp['Discriminator']['EMD'], tablefmt='latex_raw', headers='keys', floatfmt='.3'))
print(tabulate.tabulate(exps_cp['Discriminator']['HQR'], tablefmt='latex_raw', headers='keys', floatfmt='.3'))
print(tabulate.tabulate(exps_cp['Discriminator']['JSD'], tablefmt='latex_raw', headers='keys', floatfmt='.3'))

\begin{tabular}{lll}
\hline
                & WGAN, grid     & JS-GAN, 2d grid   \\
\hline
 GAN            & $0.266(0.018)$ & $0.351(0.006)$    \\
 w/o constr.    & $0.260(0.030)$ & $0.391(0.019)$    \\
 dis            & $0.264(0.040)$ & $0.409(0.008)$    \\
 cluster v0     & $0.227(0.007)$ & $0.306(0.012)$    \\
 cluster (d)    & $0.236(0.011)$ & $0.338(0.006)$    \\
 cluster v1     & $0.262(0.005)$ & $0.353(0.008)$    \\
 cluster v1 (d) & $0.241(0.008)$ & $0.364(0.005)$    \\
 cluster v2     & $0.236(0.009)$ & $0.331(0.020)$    \\
 cluster v2 (d) & $0.237(0.007)$ & $0.327(0.011)$    \\
 MMD            & $0.256(0.019)$ & $0.359(0.006)$    \\
 MMD (d)        & $0.256(0.022)$ & $0.362(0.006)$    \\
 CMD            & $0.239(0.009)$ & $0.328(0.008)$    \\
 CMD (d)        & $0.221(0.006)$ & $0.298(0.005)$    \\
\hline
\end{tabular}
\begin{tabular}{lll}
\hline
                & WGAN, grid     & JS-GAN, 2d grid   \\
\hline
 GAN            & $0.435(0.005)$ & $0.238(0.011)$    \\
 w/o constr. 

In [6]:
print(tabulate.tabulate(exps_cp['Prior']['EMD'], tablefmt='latex_raw', headers='keys', floatfmt='.3'))
print(tabulate.tabulate(exps_cp['Prior']['HQR'], tablefmt='latex_raw', headers='keys', floatfmt='.3'))
print(tabulate.tabulate(exps_cp['Prior']['JSD'], tablefmt='latex_raw', headers='keys', floatfmt='.3'))

\begin{tabular}{lll}
\hline
                & WGAN, grid     & JS-GAN, 2d grid   \\
\hline
 GAN            & $0.266(0.018)$ & $0.351(0.006)$    \\
 w/o constr.    & $0.000(0.000)$ & $0.000(0.000)$    \\
 dis            & $0.276(0.050)$ & $0.388(0.011)$    \\
 cluster v0     & $0.254(0.009)$ & $0.311(0.014)$    \\
 cluster (d)    & $0.262(0.026)$ & $0.310(0.019)$    \\
 cluster v1     & $0.276(0.008)$ & $0.338(0.012)$    \\
 cluster v1 (d) & $0.258(0.004)$ & $0.366(0.009)$    \\
 cluster v2     & $0.253(0.006)$ & $0.324(0.012)$    \\
 cluster v2 (d) & $0.246(0.004)$ & $0.345(0.015)$    \\
 MMD            & $0.256(0.006)$ & $0.350(0.009)$    \\
 MMD (d)        & $0.268(0.018)$ & $0.335(0.005)$    \\
 CMD            & $0.248(0.006)$ & $0.328(0.011)$    \\
 CMD (d)        & $0.232(0.003)$ & $0.300(0.007)$    \\
\hline
\end{tabular}
\begin{tabular}{lll}
\hline
                & WGAN, grid     & JS-GAN, 2d grid   \\
\hline
 GAN            & $0.435(0.005)$ & $0.238(0.011)$    \\
 w/o constr. 

| models   |   gan |   dumb |   dis |   clust |   clust_v1 |   clust_v1_dis |   clust_v2 |   clust_v2_dis |   mmd |   mmd_dis |   cmd |   cmd_dis |
|:---------|------:|-------:|------:|--------:|-----------:|---------------:|-----------:|---------------:|------:|----------:|------:|----------:|
| mlp_wass | 0.26  |  0.251 | 0.296 |   0.233 |      0.243 |          0.237 |      0.254 |          0.23  | 0.276 |     0.252 | 0.246 |     **0.215** |
| mlp_js   | 0.373 |  0.433 | 0.437 |   0.311 |      0.352 |          0.355 |      0.314 |          **0.301** | 0.41  |     0.36  | **0.306** |     **0.297** |
| mlp_ring | 2.11  |  2.2   | 1.52  |   2.2   |      1.82  |          **1.49**  |      2.03  |          1.94  | 2.22  |     2.22  | 2.23  |     2.15  |

| models   |   gan |   dumb |   dis |   clust |   clust_dis |   clust_v1 |   clust_v1_dis |   clust_v2 |   clust_v2_dis |   mmd |   mmd_dis |   cmd |   cmd_dis |
|:---------|------:|-------:|------:|--------:|------------:|-----------:|---------------:|-----------:|---------------:|------:|----------:|------:|----------:|
| mlp_wass | 0.254 |  0.253 | 0.318 |   0.217 |       0.251 |      0.247 |          0.238 |      0.239 |          0.221 | 0.268 |     0.258 | 0.22  |     0.231 |
| mlp_js   | 0.366 |  0.44  | 0.471 |   0.313 |       0.384 |      0.369 |          0.364 |      0.316 |          0.343 | 0.513 |     0.39  | 0.323 |     0.296 |
| mlp_ring | 2.81  |  2.81  | 2.8   |   2.81  |       2.77  |      2.03  |          1.47  |      2.37  |          2.14  | 2.81  |     2.8   | 2.81  |     2.7   |


| models   |   gan |   dumb |   dis |   clust |   clust_dis |   clust_v1 |   clust_v1_dis |   clust_v2 |   clust_v2_dis |   mmd |   mmd_dis |   cmd |   cmd_dis |
|:---------|------:|-------:|------:|--------:|------------:|-----------:|---------------:|-----------:|---------------:|------:|----------:|------:|----------:|
| mlp_wass |  0.25 |   0.25 |  0.32 |   <font color='red'>0.22</font> |        0.25 |       0.25 |           0.24 |       0.24 |           <font color='red'>0.22</font> |  0.27 |      0.26 |  <font color='red'>0.22</font> |      0.23 |
| mlp_js   |  0.37 |   0.44 |  0.47 |    <font color='red'>0.31</font> |        0.38 |       0.37 |           0.36 |       0.32 |           0.34 |  0.51 |      0.39 |  0.32 |      <font color='red'>0.3</font>  |
| mlp_ring |  2.8  |   2.8  |  2.8  |    2.8  |        2.8  |       2.0  |           <font color='red'>1.5</font>  |       2.4  |           2.2  |  2.8  |      2.8  |  2.8  |      2.4  |

clust:

$F_k(x) = \sigma(\|x - \mu_k\|_2 - \Sigma_k),~k=1..K$

clust v1:

$F_k(x) = \|x - \mu_k\|^2_2, ~k=1..K$

clust v2:

$F_k(x) = k(x, x_I) - \frac{2}{K} k(x, \mu_k) $

In [7]:
targets = ["Discriminator", "Prior", "Cond"]
models = ['mlp_ring', 'mlp_js', 'mlp_js3']
step_sizes = [0.1, 0.01, 0.01]
seeds = {
    'mlp_ring': ['', '_47', '_46'],
    'mlp_js': ['', '_47', '_46'],
    'mlp_js3': ['_44', '_47', '_46'],
}
metrics = ['EMD', 'HQR', 'JSD']

features = {
    "dumb": [""],
    #"discriminator": [""],
    "cluster": {
         "": ["",],# "_dis"],
        # "_v1": ["",],
        # "_v2": {
        #     "": ["",],

        # },

    },
    "mmd": {
            "": [""],

        },
    "cmd": {
            "": [""],
        }, 
}

In [8]:
exps_mean = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
exps_std = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

for target in targets:
    for metric in metrics:
        exps_mean[target][metric][''].append('GAN')

for target in targets:
    for model, step_size in zip(models, step_sizes):
        for metric in metrics:
            arr = []
            for seed in seeds[model]:
                path = Path(log_dir, f'dumb_feature_PriorTarget_train', f'{model}_{step_size}{seed}')
                try:
                    met = np.loadtxt(Path(path, f'{metric}.txt'))[-1]
                except (IndexError, UserWarning, OSError):
                    met = None
                if met:
                    arr.append(met)
            mean = np.mean(arr) if len(arr) > 0 else 0
            std = np.std(arr) if len(arr) > 0 else 0       
            exps_mean[target][metric][model].append(mean)
            exps_std[target][metric][model].append(std)

absent = set()
def move(subs, suff, feature):
    
    if isinstance(subs, List):
        for target in targets:
            for model, step_size in zip(models, step_sizes):
                for s in subs:
                    for metric in metrics:
                        arr = []
                        for seed in seeds[model]:
                            path = Path(log_dir, f'{feature}_feature_{target}Target_train', f'{model}{suff}{s}_{step_size}{seed}')
                            if model == models[0] and seed == seeds[model][0]:
                                exps_mean[target][metric][''].append(feature+path.name[len(model):-len(f'_{step_size}{seed}')])
                                exps_std[target][metric][''].append(feature+path.name[len(model):-len(f'_{step_size}{seed}')])

                            try:
                                met = np.loadtxt(Path(path, f'{metric}.txt'))[-1]
                            except (IndexError, UserWarning, OSError):
                                absent.add(path.as_posix())
                                met = None
                            if met:
                                arr.append(met)
                            
                        mean = np.mean(arr) if len(arr) > 0 else 0
                        std = np.std(arr) if len(arr) > 0 else 0 
                        exps_mean[target][metric][model].append(mean)
                        exps_std[target][metric][model].append(std)
        return
        
    elif isinstance(subs, Dict):
        for sub_key, subs_ in subs.items():
            move(subs_, suff+sub_key, feature)

for feature, subs in features.items():
    move(subs, "", feature)


print('\n'.join(absent))


../log/cmd_feature_CondTarget_train/mlp_js3_0.01_46
../log/cluster_feature_DiscriminatorTarget_train/mlp_js3_0.01_46
../log/mmd_feature_PriorTarget_train/mlp_js3_0.01_46
../log/mmd_feature_DiscriminatorTarget_train/mlp_js3_0.01_46


In [9]:
exps_mean_cp =  copy.deepcopy(exps_mean)
exps_std_cp =  copy.deepcopy(exps_std)

for target in targets:
    prettify(exps_mean[target], exps_mean_cp[target])
    prettify(exps_std[target], exps_std_cp[target])

exps_cp = copy.deepcopy(exps_mean_cp)

for target in targets:
    for key in metrics:
        for model in list(exps_mean_cp[target][key].keys())[1:]:
            for i, (mean, std) in enumerate(zip(exps_mean_cp[target][key][model], exps_std_cp[target][key][model])):
                exps_cp[target][key][model][i] = fr'${mean:.3f}({std:.3f})$'

In [10]:
print(tabulate.tabulate(exps_cp["Cond"]["EMD"], tablefmt='latex_raw', headers='keys', floatfmt='.3'))
print(tabulate.tabulate(exps_cp["Cond"]["HQR"], tablefmt='latex_raw', headers='keys', floatfmt='.3'))
print(tabulate.tabulate(exps_cp["Cond"]["JSD"], tablefmt='latex_raw', headers='keys', floatfmt='.3'))

\begin{tabular}{llll}
\hline
             & JS-GAN, 2d ring   & JS-GAN, 2d grid   & JS-GAN, 3d grid   \\
\hline
 GAN         & $1.044(0.203)$    & $0.405(0.035)$    & $1.439(0.038)$    \\
 w/o constr. & $0.115(0.069)$    & $0.247(0.099)$    & $0.631(0.305)$    \\
 cluster v0  & $0.182(0.233)$    & $0.224(0.085)$    & $1.460(0.170)$    \\
 MMD         & $0.108(0.087)$    & $0.237(0.082)$    & $0.926(0.384)$    \\
 CMD         & $0.304(0.217)$    & $0.295(0.071)$    & $1.191(0.120)$    \\
\hline
\end{tabular}
\begin{tabular}{llll}
\hline
             & JS-GAN, 2d ring   & JS-GAN, 2d grid   & JS-GAN, 3d grid   \\
\hline
 GAN         & $0.871(0.020)$    & $0.686(0.014)$    & $0.705(0.028)$    \\
 w/o constr. & $0.861(0.104)$    & $0.907(0.026)$    & $0.899(0.020)$    \\
 cluster v0  & $0.922(0.002)$    & $0.894(0.040)$    & $0.861(0.030)$    \\
 MMD         & $0.917(0.026)$    & $0.890(0.041)$    & $0.881(0.064)$    \\
 CMD         & $0.895(0.054)$    & $0.874(0.054)$    & $0.880(0.014)$  

In [11]:
print(tabulate.tabulate(exps_cp["Discriminator"]["EMD"], tablefmt='latex_raw', headers='keys', floatfmt='.3'))
print(tabulate.tabulate(exps_cp["Discriminator"]["HQR"], tablefmt='latex_raw', headers='keys', floatfmt='.3'))
print(tabulate.tabulate(exps_cp["Discriminator"]["JSD"], tablefmt='latex_raw', headers='keys', floatfmt='.3'))

\begin{tabular}{llll}
\hline
             & JS-GAN, 2d ring   & JS-GAN, 2d grid   & JS-GAN, 3d grid   \\
\hline
 GAN         & $1.044(0.203)$    & $0.405(0.035)$    & $1.439(0.038)$    \\
 w/o constr. & $0.185(0.235)$    & $0.466(0.037)$    & $1.425(0.064)$    \\
 cluster v0  & $0.263(0.180)$    & $0.506(0.035)$    & $1.536(0.066)$    \\
 MMD         & $0.282(0.184)$    & $0.435(0.140)$    & $1.345(0.135)$    \\
 CMD         & $0.599(0.124)$    & $0.391(0.046)$    & $1.522(0.042)$    \\
\hline
\end{tabular}
\begin{tabular}{llll}
\hline
             & JS-GAN, 2d ring   & JS-GAN, 2d grid   & JS-GAN, 3d grid   \\
\hline
 GAN         & $0.871(0.020)$    & $0.686(0.014)$    & $0.705(0.028)$    \\
 w/o constr. & $0.846(0.055)$    & $0.825(0.052)$    & $0.832(0.029)$    \\
 cluster v0  & $0.902(0.010)$    & $0.829(0.033)$    & $0.913(0.011)$    \\
 MMD         & $0.304(0.422)$    & $0.833(0.034)$    & $0.806(0.071)$    \\
 CMD         & $0.927(0.016)$    & $0.828(0.051)$    & $0.830(0.017)$  

In [12]:
print(tabulate.tabulate(exps_cp["Prior"]["EMD"], tablefmt='latex_raw', headers='keys', floatfmt='.3'))
print(tabulate.tabulate(exps_cp["Prior"]["HQR"], tablefmt='latex_raw', headers='keys', floatfmt='.3'))
print(tabulate.tabulate(exps_cp["Prior"]["JSD"], tablefmt='latex_raw', headers='keys', floatfmt='.3'))

\begin{tabular}{llll}
\hline
             & JS-GAN, 2d ring   & JS-GAN, 2d grid   & JS-GAN, 3d grid   \\
\hline
 GAN         & $1.044(0.203)$    & $0.405(0.035)$    & $1.439(0.038)$    \\
 w/o constr. & $1.044(0.203)$    & $0.405(0.035)$    & $1.439(0.038)$    \\
 cluster v0  & $0.119(0.100)$    & $0.225(0.109)$    & $1.499(0.143)$    \\
 MMD         & $0.144(0.080)$    & $0.241(0.076)$    & $0.948(0.414)$    \\
 CMD         & $0.231(0.056)$    & $0.170(0.057)$    & $0.423(0.036)$    \\
\hline
\end{tabular}
\begin{tabular}{llll}
\hline
             & JS-GAN, 2d ring   & JS-GAN, 2d grid   & JS-GAN, 3d grid   \\
\hline
 GAN         & $0.871(0.020)$    & $0.686(0.014)$    & $0.705(0.028)$    \\
 w/o constr. & $0.871(0.020)$    & $0.686(0.014)$    & $0.705(0.028)$    \\
 cluster v0  & $0.942(0.010)$    & $0.865(0.017)$    & $0.868(0.027)$    \\
 MMD         & $0.924(0.017)$    & $0.886(0.036)$    & $0.910(0.005)$    \\
 CMD         & $0.935(0.008)$    & $0.883(0.024)$    & $0.930(0.008)$  

In [14]:
# exps_mean = defaultdict(lambda: defaultdict(list))

# metrics = ['EMD', 'HQR', 'JSD']

# exps_mean['EMD'][''].append('GAN')
# exps_mean['HQR'][''].append('GAN')
# exps_mean['JSD'][''].append('GAN')

# for model, step_size in zip(models, step_sizes):

#     path = Path(log_dir, f'dumb_feature_PriorTarget_train', f'{model}_{step_size}')
#     try:
#         emd = np.loadtxt(Path(path, 'EMD.txt'))[-1] #[:, 0]
#         hqr = np.loadtxt(Path(path, 'HQR.txt'))[-1]
#         jsd = np.loadtxt(Path(path, 'JSD.txt'))[-1]
#     except:
#         emd, hqr, jsd = 0, 0, 0

#     exps_mean['EMD'][model].append(emd)
#     exps_mean['HQR'][model].append(hqr)
#     exps_mean['JSD'][model].append(jsd)

# def move(subs, suff, feature):
#     if isinstance(subs, List):
#         for model, step_size in zip(models, step_sizes):
#             for s in subs:
#                 path = Path(log_dir, f'{feature}_feature_{target}Target_train', f'{model}{suff}{s}_{step_size}')
#                 try:
#                     emd = np.loadtxt(Path(path, 'EMD.txt'))[-1] #[:, 0]
#                     hqr = np.loadtxt(Path(path, 'HQR.txt'))[-1]
#                     jsd = np.loadtxt(Path(path, 'JSD.txt'))[-1]
#                 except:
#                     emd, hqr, jsd = 0, 0, 0

#                 exps_mean['EMD'][model].append(emd)
#                 exps_mean['HQR'][model].append(hqr)
#                 exps_mean['JSD'][model].append(jsd)
#                 #exps_std[model].append(std)
#                 if model == models[0]:
#                     exps_mean['EMD'][''].append(feature+path.name[len(model):-len(f'_{step_size}')])
#                     exps_mean['HQR'][''].append(feature+path.name[len(model):-len(f'_{step_size}')])
#                     exps_mean['JSD'][''].append(feature+path.name[len(model):-len(f'_{step_size}')])
        
#             # exps.extend([Path(log_dir, f'{feature}_feature_{target}Target', f'{model}{suff}{s}_0.001') for s in subs])
#         return
#     elif isinstance(subs, Dict):
#         for sub_key, subs_ in subs.items():
#             move(subs_, suff+sub_key, feature)

# for feature, subs in features.items():
#     move(subs, "", feature)


In [15]:
# import copy

# def prettify(exps_mean, exps_cp):
#     for key in metrics:
#         exps_cp[key][''] = []
#         for k in exps_mean[key]['']:
#             k = k.replace('discriminator', 'dis')
#             k = k.replace('mmd', 'MMD')
#             k = k.replace('cmd', 'CMD')
#             k = k.replace('_dis', ' (d)')
#             k = k.replace('_', ' ')
#             exps_cp[key][''].append(k)


#         for model in models:
#             exps_cp[key].pop(model)
#             for i, mean in enumerate(exps_mean[key][model]):#, exps_std[model])):
#                 # if exps_cp[key].get(model):
#                 #     exps_cp[key].pop(model)
#                 if model == 'mlp_ring':
#                     model_ = 'JS-GAN, 2d ring'
#                 elif model == 'mlp_js':
#                     model_ = 'JS-GAN, 2d grid'
#                 elif model == 'mlp_wass':
#                     model_ = 'WGAN, 2d grid'
#                 exps_cp[key][model_].append(fr'${mean:.3f}$') #({std:.3f})$'
#                # dictionary[new_key] = dictionary.pop(old_key)


# exps_mean_cp =  copy.deepcopy(exps_mean)

# prettify(exps_mean, exps_mean_cp)


In [16]:
# print(tabulate.tabulate(exps_mean_cp["EMD"], tablefmt='latex_raw', headers='keys', floatfmt='.3'))

In [17]:
# models = ['mlp_ring', 'mlp_js', 'mlp_wass']

# step_sizes = [0.1, 0.01, 0.01]

# features = {
#     "dumb": [""],
#     #"discriminator": [""],
#     "cluster": {
#          "": ["",],# "_dis"],
#         "_v1": ["",],
#         "_v2": {
#             "": ["",],
#             # "_lin": ["", "_dis"],
#             # "_quad": ["", "_dis"],
#         },

#     },
#     "mmd": {
#             "": [""],
#             # "_lin": [""], #"_dis"],
#             # "_quad": [""],#, "_dis"],
#         },
#     "cmd": {
#             "": [""],
#         }, 
# }
# target = "Cond"

# import tabulate
# from pathlib import Path
# import numpy as np

# from typing import List, Dict
# from collections import defaultdict

# import tabulate

# log_dir = '../log'

In [18]:
# exps_mean = defaultdict(lambda: defaultdict(list))

# metrics = ['EMD', 'HQR', 'JSD']

# exps_mean['EMD'][''].append('GAN')
# exps_mean['HQR'][''].append('GAN')
# exps_mean['JSD'][''].append('GAN')

# for model, step_size in zip(models, step_sizes):

#     path = Path(log_dir, f'dumb_feature_PriorTarget_train', f'{model}_{step_size}')
#     try:
#         emd = np.loadtxt(Path(path, 'EMD.txt'))[-1] #[:, 0]
#         hqr = np.loadtxt(Path(path, 'HQR.txt'))[-1]
#         jsd = np.loadtxt(Path(path, 'JSD.txt'))[-1]
#     except:
#         emd, hqr, jsd = 0, 0, 0

#     exps_mean['EMD'][model].append(emd)
#     exps_mean['HQR'][model].append(hqr)
#     exps_mean['JSD'][model].append(jsd)

# def move(subs, suff, feature):
#     if isinstance(subs, List):
#         for model, step_size in zip(models, step_sizes):
#             for s in subs:
#                 path = Path(log_dir, f'{feature}_feature_{target}Target_train', f'{model}{suff}{s}_{step_size}')
#                 try:
#                     emd = np.loadtxt(Path(path, 'EMD.txt'))[-1] #[:, 0]
#                     hqr = np.loadtxt(Path(path, 'HQR.txt'))[-1]
#                     jsd = np.loadtxt(Path(path, 'JSD.txt'))[-1]
#                 except:
#                     emd, hqr, jsd = 0, 0, 0

#                 exps_mean['EMD'][model].append(emd)
#                 exps_mean['HQR'][model].append(hqr)
#                 exps_mean['JSD'][model].append(jsd)
#                 #exps_std[model].append(std)
#                 if model == models[0]:
#                     exps_mean['EMD'][''].append(feature+path.name[len(model):-len(f'_{step_size}')])
#                     exps_mean['HQR'][''].append(feature+path.name[len(model):-len(f'_{step_size}')])
#                     exps_mean['JSD'][''].append(feature+path.name[len(model):-len(f'_{step_size}')])
        
#             # exps.extend([Path(log_dir, f'{feature}_feature_{target}Target', f'{model}{suff}{s}_0.001') for s in subs])
#         return
#     elif isinstance(subs, Dict):
#         for sub_key, subs_ in subs.items():
#             move(subs_, suff+sub_key, feature)

# for feature, subs in features.items():
#     move(subs, "", feature)


In [19]:
# import copy

# def prettify(exps_mean, exps_cp):
#     for key in metrics:
#         exps_cp[key][''] = []
#         for k in exps_mean[key]['']:
#             k = k.replace('discriminator', 'dis')
#             k = k.replace('mmd', 'MMD')
#             k = k.replace('cmd', 'CMD')
#             k = k.replace('_dis', ' (d)')
#             k = k.replace('_', ' ')
#             exps_cp[key][''].append(k)


#         for model in models:
#             exps_cp[key].pop(model)
#             for i, mean in enumerate(exps_mean[key][model]):#, exps_std[model])):
#                 # if exps_cp[key].get(model):
#                 #     exps_cp[key].pop(model)
#                 if model == 'mlp_ring':
#                     model_ = 'JS-GAN, 2d ring'
#                 elif model == 'mlp_js':
#                     model_ = 'JS-GAN, 2d grid'
#                 elif model == 'mlp_wass':
#                     model_ = 'WGAN, 2d grid'
#                 exps_cp[key][model_].append(fr'${mean:.3f}$') #({std:.3f})$'
#                # dictionary[new_key] = dictionary.pop(old_key)


# exps_mean_cp =  copy.deepcopy(exps_mean)

# prettify(exps_mean, exps_mean_cp)


In [20]:
# print(tabulate.tabulate(exps_mean_cp["EMD"], tablefmt='latex_raw', headers='keys', floatfmt='.3'))

In [21]:
# print(tabulate.tabulate(exps_mean_cp["HQR"], tablefmt='latex_raw', headers='keys', floatfmt='.3'))

In [22]:
# print(tabulate.tabulate(exps_mean_cp["JSD"], tablefmt='latex_raw', headers='keys', floatfmt='.3'))